In [1]:
pip install requests beautifulsoup4 pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Function to scrape stock price from Google Finance
def scrape_google_finance(stock_symbol):
    url = f'https://www.google.com/finance/quote/{stock_symbol}:NASDAQ'
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"  # To mimic a browser request
    }
    
    # Send a GET request to the URL
    response = requests.get(url, headers=headers)
    
    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve data for {stock_symbol}. Status code: {response.status_code}")
        return None
    
    # Parse the webpage content
    soup = BeautifulSoup(response.text, 'html.parser')

    try:
        # Extract the stock price
        price_tag = soup.find('div', class_='YMlKec fxKbKc')
        if not price_tag:
            print(f"Price tag not found for {stock_symbol}.")
            return None
        price = price_tag.text.replace('$', '').replace(',', '').strip()

        # Create a dictionary to store stock data
        stock_data = {'Stock Symbol': stock_symbol, 'Current Price': price}
        return stock_data
    
    except Exception as e:
        print(f"Error parsing data for {stock_symbol}: {e}")
        return None

# Function to scrape multiple stock symbols
def scrape_multiple_stocks(stock_symbols, delay=1):
    all_stock_data = []
    
    for symbol in stock_symbols:
        print(f"Scraping data for {symbol}...")
        data = scrape_google_finance(symbol)
        if data:
            all_stock_data.append(data)
        time.sleep(delay)  # Avoid hitting the server too hard
    
    # Convert list of dictionaries to DataFrame
    stock_df = pd.DataFrame(all_stock_data)
    return stock_df

# Example usage
if __name__ == "__main__":
    stock_symbols = ["AAPL", "GOOGL", "MSFT", "AMZN", "TSLA"]
    stock_df = scrape_multiple_stocks(stock_symbols)
    
    if not stock_df.empty:
        stock_df.to_csv("google_stock_data.csv", index=False)
        print(stock_df)


Scraping data for AAPL...
Scraping data for GOOGL...
Scraping data for MSFT...
Scraping data for AMZN...
Scraping data for TSLA...
  Stock Symbol Current Price
0         AAPL        227.73
1        GOOGL        163.95
2         MSFT        428.02
3         AMZN        187.99
4         TSLA        260.46
